In [1]:
## Authors:     Michael Quillen & Max Parker; M.D. Candidates @ University of Florida
## Project:     'Towards prediction of CRC in patients under the age of 50'
## PIs:         Dr. Thomas George, MD; Dr. Jiang Bian, PhD
## 
## **base code adapted from Dr. Xi Yang, PhD project: 'Early Prediction of Alzheimer's Disease and Related Dementias
##                                                          Using Electronic Health Records'

%load_ext autoreload
%autoreload 2

import os
import timeit
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
from functools import partial, reduce
from sklearn import preprocessing
from IPython.display import clear_output

InteractiveShell.ast_node_interactivity = "all"

os.chdir('/mnt/data1/songziwu/data/crc_data/')


In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
def pkl_dump(data,file):
    with open(file,"wb") as fw:
        pkl.dump(data,fw)
        
def pkl_load(file):
    with open(file,"rb") as fr:
        data = pkl.load(fr)
    return data



In [4]:
# function of cross table frequency and column percentage
def crosstable_oc(df,row,column,round=4):
    crosstable = pd.crosstab(index=df[row],columns = df[column], margins = True)
    crosstable.index.values[-1] = 'coltotal'
    #column percentage
    col_p = np.round(crosstable / crosstable.loc["coltotal"],round)*100
    
    
    print('\n')
    print('Frequency')
    print(crosstable)
    print('\n')
    print('Column%')
    print(col_p)

In [5]:
tempdata_path = "TempData1"

In [6]:
os.getcwd()
os.chdir('/mnt/data1/songziwu/data/crc_data/')
os.getcwd()

'/mnt/data1/songziwu/data/crc_data'

'/mnt/data1/songziwu/data/crc_data'

# Load Outcomes

In [7]:
df_otc = pd.read_csv('Outcome.csv', dtype = str)
df_otc.head()
df_otc.shape

,PATID,Outcome
0,11e750609b98b9dcbf840050569ea8fb,1
1,11e750609bdcba56a7850050569ea8fb,1
2,11e75060a10b8656baf60050569ea8fb,1
3,11e75060a83276b085f80050569ea8fb,1
4,11e75060aae36aa488ed0050569ea8fb,1


(35785, 2)

In [8]:
#print(df_otc.iloc[[20000]])
#print(df_otc.iloc[[1]])

# Load Encounters

In [9]:
df_enc = pd.read_csv('ENCOUNTER.csv', dtype = str)
df_enc.head()
df_enc.shape

,ENCOUNTERID,PATID,ADMIT_DATE,ADMIT_TIME,DISCHARGE_DATE,DISCHARGE_TIME,PROVIDERID,FACILITY_LOCATION,ENC_TYPE,FACILITYID,DISCHARGE_DISPOSITION,DISCHARGE_STATUS,DRG,DRG_TYPE,ADMITTING_SOURCE,PAYER_TYPE_PRIMARY,PAYER_TYPE_SECONDARY,FACILITY_TYPE,RAW_SITEID,RAW_ENC_TYPE,RAW_DISCHARGE_DISPOSITION,RAW_DISCHARGE_STATUS,RAW_DRG_TYPE,RAW_ADMITTING_SOURCE,RAW_FACILITY_TYPE,RAW_PAYER_TYPE_PRIMARY,RAW_PAYER_NAME_PRIMARY,RAW_PAYER_ID_PRIMARY,RAW_PAYER_TYPE_SECONDARY,RAW_PAYER_NAME_SECONDARY,RAW_PAYER_ID_SECONDARY,UPDATED,SOURCE
0,a+xOB8oBTFCu3wa/ePVtJjR9uvkS+jd1znFMUN+oBM0J8x...,11e75061a5e8c7c8a3fa0050569ea8fb,2013-04-19,08:15,2013-04-25,13:43,a+xOc7gHSlar,327,IP,G2,A,HO,885,01,OT,2,81,HOSPITAL_PSYCHIATRIC,NaN,NaN,1,01,1213 CMS DRG LIST,1,G2,MEDICAID,MEDICAID OF FLORIDA,20010,SELF PAY,COMM CARE PENDING FOLLOW-UP,01084,2018-09-25 10:49:07.000,ORL
1,a+xOB8sESVDYogO+e/YeJkV8uYkWijEFsAo4UqCjdLx+9m...,11e7506100fa593488ed0050569ea8fb,2017-09-05,22:25,2017-09-06,01:57,a+xOc7gDQ1Wt,328,ED,B1,A,AM,000,NaN,OT,511,81,HOSPITAL_COMMUNITY,NaN,NaN,7,07,NaN,1,B1,HMO,WELLCARE/STAYWEL/HLTHEAS MCAID,20034,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:49:57.000,ORL
2,a+xOB8txQlLY2wS8fflvVEl8y4odi0UAzgc+IK7fBLsK9x...,11e75061216d13e68cac0050569ea8fb,2014-02-15,14:42,2014-02-19,18:57,a+xOc7gHQ1+p,328,IP,M2,A,SN,603,01,OT,1,2,HOSPITAL_BASED_OUTPATIENT_CLINIC_OR_DEPARTMENT...,NaN,NaN,3,03,1314 CMS DRG LIST,1,M2,MEDICARE,MEDICARE,25010,MEDICAID,MEDICAID OF FLORIDA,20010,2018-09-25 10:49:56.000,ORL
3,a+xOB8wDTVCp2A24eoRoJjN8y/xl+DUGuAFNJazYdsxy+B...,11e7506101fb9c9e9be90050569ea8fb,2012-08-21,21:02,2012-08-22,09:15,a+xOc7gKSFKg,327,ED,G1,A,HO,000,NaN,OT,511,81,HOSPITAL_COMMUNITY,NaN,NaN,1,01,NaN,1,G1,HMO,AMERIGROUP HMO MEDICAID,20066,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:48:20.000,ORL
4,a+xOBb0BT1esoge8ffloJ0MOuIwQ+z53zQU+J6usDMl79R...,11e75060c0e2203e94360050569ea8fb,2018-08-02,18:11,2018-08-02,21:06,a+xOc7gCQlSh,328,ED,A1,A,HO,000,NaN,OT,511,81,OT,NaN,NaN,1,01,NaN,1,A1,HMO,WELLCARE/STAYWEL/HLTHEAS MCAID,20034,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:51:56.000,ORL


(7964989, 33)

In [10]:
# ADD CRC STATUS FILE
def addCRCst(df,otc):
    df = df.merge(otc,left_on='PATID',right_on='PATID')
    df.rename(columns={'Outcome':'CRC_STATUS'},inplace=True)
    return df

In [11]:
df_enc = addCRCst(df_enc,df_otc)

In [12]:
df_enc.head()
df_enc.shape
#print(df_enc.loc[df_enc.PATID=='11e82174ff5584d8afa90050569ea8fb'])
#print(df_enc.loc[df_enc.PATID=='11e750609bdcba56a7850050569ea8fb'])

,ENCOUNTERID,PATID,ADMIT_DATE,ADMIT_TIME,DISCHARGE_DATE,DISCHARGE_TIME,PROVIDERID,FACILITY_LOCATION,ENC_TYPE,FACILITYID,DISCHARGE_DISPOSITION,DISCHARGE_STATUS,DRG,DRG_TYPE,ADMITTING_SOURCE,PAYER_TYPE_PRIMARY,PAYER_TYPE_SECONDARY,FACILITY_TYPE,RAW_SITEID,RAW_ENC_TYPE,RAW_DISCHARGE_DISPOSITION,RAW_DISCHARGE_STATUS,RAW_DRG_TYPE,RAW_ADMITTING_SOURCE,RAW_FACILITY_TYPE,RAW_PAYER_TYPE_PRIMARY,RAW_PAYER_NAME_PRIMARY,RAW_PAYER_ID_PRIMARY,RAW_PAYER_TYPE_SECONDARY,RAW_PAYER_NAME_SECONDARY,RAW_PAYER_ID_SECONDARY,UPDATED,SOURCE,CRC_STATUS
0,a+xOB8oBTFCu3wa/ePVtJjR9uvkS+jd1znFMUN+oBM0J8x...,11e75061a5e8c7c8a3fa0050569ea8fb,2013-04-19,08:15,2013-04-25,13:43,a+xOc7gHSlar,327,IP,G2,A,HO,885,01,OT,2,81,HOSPITAL_PSYCHIATRIC,NaN,NaN,1,01,1213 CMS DRG LIST,1,G2,MEDICAID,MEDICAID OF FLORIDA,20010,SELF PAY,COMM CARE PENDING FOLLOW-UP,01084,2018-09-25 10:49:07.000,ORL,0
1,a+xOAs1zOVWg2XTKDfkWKDcOzPxihz5zsXc9X66vDLkNgm...,11e75061a5e8c7c8a3fa0050569ea8fb,2014-12-23,20:39,2014-12-24,04:24,a+xOc7gCQlCh,328,ED,A1,A,HO,000,NaN,OT,511,81,OT,NaN,NaN,1,01,NaN,1,A1,HMO,SIMPLY CLEAR HEALTH ALLIANCE,20084,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:50:47.000,ORL,0
2,a+xOArgGPVfb2XC5CfFtVDJ7yIoR/DRxzgY4JKiicL1y9h...,11e75061a5e8c7c8a3fa0050569ea8fb,2014-08-27,19:40,2014-08-27,22:28,a+xOc7gDQ1eq,328,ED,A1,A,HO,000,NaN,OT,511,81,OT,NaN,NaN,1,01,NaN,1,A1,HMO,SIMPLY CLEAR HEALTH ALLIANCE,20084,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:50:29.000,ORL,0
3,a+xOAbAKQ1SqqHS9D4cfVjIPyopiiUcGugY5ItivDboP9x...,11e75061a5e8c7c8a3fa0050569ea8fb,2012-06-03,12:25,2012-06-03,15:30,a+xOc7gDTFev,328,ED,A1,A,HO,000,NaN,OT,2,81,OT,NaN,NaN,1,01,NaN,1,A1,MEDICAID,ZMEDIPASS STATE MEDICAID HMO,20012,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:48:04.000,ORL,0
4,a+xOc74KTyesowa5D/MdI0J/yP9ijzB6uwc6J9qpd8oPgx...,11e75061a5e8c7c8a3fa0050569ea8fb,2015-05-31,03:23,2015-06-01,13:04,a+xOc7gHT1Sr,327,IP,G2,A,HO,885,01,OT,511,81,HOSPITAL_PSYCHIATRIC,NaN,NaN,1,01,1415 CMS DRG LIST,1,G2,HMO,BEH SIMPLY CHA BEACON HS MCAID,20102,SELF PAY,SP AFTER INSURANCE,01004,2018-09-25 10:51:19.000,ORL,0


(7964989, 34)

In [13]:
df_enc.groupby('ENC_TYPE').size()
df_enc.groupby('SOURCE').size()
crosstable_oc(df_enc,'SOURCE','ENC_TYPE')

ENC_TYPE
AV    3053476
ED     361407
EI      55757
IC    1291726
IP      33015
IS       1091
NI       2169
OA      41168
OS      62768
OT    2915664
UN     146748
dtype: int64

SOURCE
AVH     145517
BND        468
CHP      10877
FLM    6971418
NCH       1505
ORL      22197
TMA      42141
TMC      15197
UFH     617952
UMI     137717
dtype: int64



Frequency
ENC_TYPE       AV      ED     EI       IC     IP    IS    NI     OA     OS  \
SOURCE                                                                       
AVH         61142   28789   7181        0   2442   256    46  38746   4199   
BND           468       0      0        0      0     0     0      0      0   
CHP          6285     198     58     1800     96     0     0      0     59   
FLM       2454259  300413  43805  1289917  23437   823     0      0  57090   
NCH          1254     146      0        0     59     0     0      0     17   
ORL         11293    5672      0        0   2208     0     0   2347    677   
TMA         42141       0      0        0      0     0     0      0      0   
TMC          4018    5074      0        9   1417    12  1895      0    338   
UFH        335217   21103   4713        0   3355     0     0      0    386   
UMI        137399      12      0        0      1     0   228     75      2   
coltotal  3053476  361407  55757  1291726  33015  10

# Diagnosis Load

In [14]:
df_dx = pd.read_csv('DIAGNOSIS.csv',dtype = str)
df_dx = addCRCst(df_dx,df_otc)
df_dx.head()
df_dx.shape


,DIAGNOSISID,PATID,ENCOUNTERID,ENC_TYPE,ADMIT_DATE,PROVIDERID,DX,DX_TYPE,DX_DATE,DX_SOURCE,DX_ORIGIN,PDX,DX_POA,RAW_DX,RAW_DX_TYPE,RAW_DX_SOURCE,RAW_PDX,RAW_DX_POA,UPDATED,SOURCE,RAW_DX_ORIGIN,CRC_STATUS
0,a+xOc80CSFSp2ATJe/gWUTQIvI1nikUGy3FMIt+rALxy8h...,11e75060feeb8046ab480050569ea8fb,a+xOes0FTV6sqQe/D/cZVkN4y/gdijICvQpMVKyjc755hR...,ED,2016-07-28,a+xOc7gGTFeh,G43.909,10,NaN,DI,BI,S,NaN,G43.909,ICD10,DI,3,NaN,2018-09-25 10:56:30.000,ORL,BI,0
1,a+xOc7oCSFKuqwbMefFsJUJ/zY5ljD91unNIX6mocbFy8m...,11e75060feeb8046ab480050569ea8fb,a+xOd75xOCLa2wC6CoUWJklysfsWiD53v3M9IKnbALAO8W...,ED,2013-07-16,a+xOc7gKSVaq,525.9,09,NaN,DI,BI,P,NaN,525.9,ICD9,DI,1,NaN,2018-09-25 11:00:22.000,ORL,BI,0
2,a+xOc7l3Ol7aowy6ePAXUTN+yv0SjEcAygJMVt3ZDMoI9R...,11e75060feeb8046ab480050569ea8fb,a+xOdr8HOlbYrgy8cvYfJ0N6zItnj0RzzQs4X9/YA7wJ9W...,ED,2018-08-08,a+xOc7gFTFKh,Z3A.34,10,NaN,DI,BI,S,NaN,Z3A.34,ICD10,DI,3,NaN,2018-09-25 11:08:15.000,ORL,BI,0
3,a+xOc7kATSWg2QLKDvIZIkkPv/Zh/TBwygo6UN3ZdLsIgh...,11e75060feeb8046ab480050569ea8fb,a+xOcbkKTFbdrXGwevMfUkMLsPpgiz90znM+Iq3fDLh7+W...,ED,2017-10-08,a+xOc7gDTFev,K08.89,10,NaN,DI,BI,P,NaN,K08.89,ICD10,DI,1,NaN,2018-09-25 10:54:54.000,ORL,BI,0
4,a+xOc7hzPiWq3AS6fPBsUkJ6vvdmizFwunQ5Va7bDL8N+R...,11e75060feeb8046ab480050569ea8fb,a+xOcLAGTCShrXTOfPkaKUN7uf9gjDN1zQJCXqHbDbwK8G...,ED,2016-01-08,a+xOc7gKT1Gu,I10,10,NaN,DI,BI,S,NaN,I10,ICD10,DI,2,NaN,2018-09-25 10:54:24.000,ORL,BI,0


(16212826, 22)

In [15]:
# format the date
df_dx['ADMIT_DATE'] = df_dx['ADMIT_DATE'].astype('datetime64[ns]')

In [16]:
df_dx.PATID.nunique()
df_dx.ENCOUNTERID.nunique()
df_enc.shape[0]
df_enc.ENCOUNTERID.nunique()

35768

6734457

7964989

7964989

In [17]:
df_dx.groupby("CRC_STATUS").PATID.nunique()

CRC_STATUS
0    34337
1     1431
Name: PATID, dtype: int64

In [18]:
df_dx.groupby('DX_SOURCE').size()

DX_SOURCE
AD      226894
DI     2187673
FI    13647297
IN       30217
OT      120745
dtype: int64

In [19]:
os.chdir('/mnt/data1/songziwu/data/crc_data/01_MQ_Incident_Match_Files/')
preIMcontrols = pd.read_csv("preIMcontrols.csv")
preIMcasesCRC = pd.read_csv("preIMcasesCRC.csv")
preIMcasesCC = pd.read_csv("preIMcasesCC.csv")
preIMcasesRC = pd.read_csv("preIMcasesRC.csv")
# os.chdir('/mnt/data/shared/crc_under_50/DATA_v2/')

In [20]:
#preIMcasesCRC

In [21]:
#################################################
#################################################

In [22]:
#df_dx_CRC = df_dx.loc[df_dx.DX.str.replace(".","").isin(CRC_ICD_CODES)]
#df_dx_CC = df_dx.loc[df_dx.DX.str.replace(".","").isin(CC_ICD)]
#df_dx_RC = df_dx.loc[df_dx.DX.str.replace(".","").isin(RC_ICD)]


In [23]:
#df_dx_CRC.head()
#df_dx_CRC.shape
#df_dx_CRC.groupby('CRC_STATUS').PATID.nunique()
#df_dx_CC.head()
#df_dx_CC.shape
#df_dx_CC.groupby('CRC_STATUS').PATID.nunique()
#df_dx_RC.head()
#df_dx_RC.shape
#df_dx_RC.groupby('CRC_STATUS').PATID.nunique()

In [24]:
#df_dx_CRC[df_dx_CRC.CRC_STATUS == 0].DX.unique()
#df_dx_CC[df_dx_CC.CRC_STATUS == 0].DX.unique()
#df_dx_RC[df_dx_RC.CRC_STATUS == 0].DX.unique()

In [25]:
#df_dx_CRC.groupby('ENC_TYPE').size()
#df_dx_CRC.groupby('SOURCE').size()
#df_dx_CC.groupby('ENC_TYPE').size()
#df_dx_CC.groupby('SOURCE').size()
#df_dx_RC.groupby('ENC_TYPE').size()
#df_dx_RC.groupby('SOURCE').size()

In [26]:
#df_dx_CRC.groupby('DX_SOURCE').size()
#df_dx_CRC.groupby('DX_ORIGIN').size()
##df_dx_CRC.groupby('PDX').size()
#df_dx_CC.groupby('DX_SOURCE').size()
#df_dx_CC.groupby('DX_ORIGIN').size()
#df_dx_CC.groupby('PDX').size()
#df_dx_RC.groupby('DX_SOURCE').size()
#df_dx_RC.groupby('DX_ORIGIN').size()
#df_dx_RC.groupby('PDX').size()

In [27]:
#df_dx_CRC[df_dx_CRC.DX_SOURCE == 'AD'].head()
#df_dx_CC[df_dx_CC.DX_SOURCE == 'AD'].head()
#df_dx_RC[df_dx_RC.DX_SOURCE == 'AD'].head()

In [28]:
#df_dx_CRC[df_dx_CRC.ENCOUNTERID == 'cfhOdLEBTVWurw==']

In [29]:
#df_dx_CRC.PATID.nunique()
#df_dx_CRC[df_dx_CRC.DX_SOURCE != 'AD'].PATID.nunique()
#df_dx_CC.PATID.nunique()
#df_dx_CC[df_dx_CC.DX_SOURCE != 'AD'].PATID.nunique()
#df_dx_RC.PATID.nunique()
#df_dx_RC[df_dx_RC.DX_SOURCE != 'AD'].PATID.nunique()

In [30]:
#crosstable_oc(df_dx_CRC,'DX_SOURCE','DX_ORIGIN')
#crosstable_oc(df_dx_CC,'DX_SOURCE','DX_ORIGIN')
#crosstable_oc(df_dx_RC,'DX_SOURCE','DX_ORIGIN')


In [31]:
#############################################################
##############################################################

In [32]:
## TESTING OF 2nd Encounter extraction ##

#_df_2 = df_dx.groupby('PATID')['ADMIT_DATE'].apply(lambda x: x.drop_duplicates().nsmallest(2))
#_df_2.head()
#_df_2 = _df_2.reset_index()
#_df_2.head()
#_df_2.shape
#type(_df_2)
#idx2 = _df_2.level_1.loc[_df_2.groupby('PATID')['ADMIT_DATE'].idxmax()]
#mask = df_dx.index.isin(idx2)
#_df_2 = df_dx[mask]
#_df_2.head()
#_df_2.shape
#_df_2 = _df_2[['PATID','ADMIT_DATE','DX']]
#_df_2.head()

In [33]:
def extract_history(df, criteriacode, criteriavar, idvar, datevar, returnvar, extractname):
    # keep data rows meet the criteria
    _df_meetcriteria = df.loc[df[criteriavar].str.replace(".","").isin(criteriacode)]

    # create df with unique patient
    _df_uniquepatid = pd.DataFrame({idvar: df[idvar].unique()})
    # create indicator of ever has the criteriacode
    _df_uniquepatid['has_'+extractname] = _df_uniquepatid[idvar].isin(_df_meetcriteria[idvar]).astype(int)

    # obtain the last record for each patient with criteria code
    _df_firstrecord = _extract_first_record(_df_meetcriteria, idvar, datevar, returnvar, extractname)

    # merge the tables
    _df_list = [_df_uniquepatid, _df_firstrecord]
    _df_out = reduce(lambda left, right: pd.merge(left, right, on=[idvar], how='outer'), _df_list)
    return _df_out

def _extract_2nd_record(df_long, idvar, datevar, returnvar, extractname):
    ''' this function return the last date for each idvar for long format df '''
    _df_2 = df_long.groupby(idvar)[datevar].apply(lambda x: x.drop_duplicates().nsmallest(2)).reset_index()
    idx2 = _df_2.level_1.loc[_df_2.groupby(idvar)[datevar].idxmax()]                     
    mask2 = df_long.index.isin(idx2)
    _df_2 = df_long[mask2]
    _df_2 = _df_2[[idvar,datevar,returnvar]]
    _df_2.rename(columns={datevar:extractname + '_2nd_date', returnvar:extractname + '_2nd_code'}, inplace=True)
    return _df_2

def _extract_first_record(df_long, idvar, datevar, returnvar, extractname):
    ''' this function return the last date for each idvar for long format df '''
    _df_f = df_long.loc[df_long.groupby(idvar)[datevar].idxmin(), [idvar,datevar,returnvar]]
    _df_f.rename(columns={datevar:extractname + '_f_date', returnvar:extractname + '_f_code'}, inplace=True)
    return _df_f

In [34]:
CRC_ICD_CODES = ['C180','C181','C182','C183','C184','C185','C186',
          'C187','C188','C189','1530','1531','1532','1533','1534',
          '1535','1536','1537','1538','1539','Z85038','D010',
          '2303','V1005','C19','C20','1540','1541','1548','D012','D011',
          '2304','Z85048','V1006']

CC_ICD = ['C180','C181','C182','C183','C184','C185','C186',
          'C187','C188','C189','1530','1531','1532','1533','1534',
          '1535','1536','1537','1538','1539','Z85038','D010',
          '2303','V1005']

RC_ICD = ['C19','C20','1540','1541','1548','D012','D011',
          '2304','Z85048','V1006']

In [35]:
# all CRC diagnoses
df_CRC = extract_history(df_dx, CRC_ICD_CODES,'DX','PATID','ADMIT_DATE','DX','CRC')
df_CRC.head()
df_CRC.shape

df_CRC.groupby('has_CRC').PATID.size()
df_CRC.groupby('CRC_f_code').PATID.size().sort_values(0,ascending=False)

df_CRC[df_CRC.has_CRC == 1].head()

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75060feeb8046ab480050569ea8fb,0,NaT,NaN
1,11e7506106e83532bb6a0050569ea8fb,0,NaT,NaN
2,11e75060d2b34f18907a0050569ea8fb,0,NaT,NaN
3,11e75060deaca97c845d0050569ea8fb,0,NaT,NaN
4,11e75060b2681860ad370050569ea8fb,0,NaT,NaN


(35768, 4)

has_CRC
0    34308
1     1460
Name: PATID, dtype: int64

CRC_f_code
C18.9      310
C20        197
153.9      145
C18.7      110
Z85.038     92
V10.05      79
Z85.048     55
154.1       51
C18.1       50
C18.2       46
C19         46
C18.0       33
C18.4       32
154.0       29
C18.6       26
V10.06      26
153.5       18
C18.3       16
153.4       13
153.3       13
C18.8        9
153.1        8
D01.0        7
153.6        7
153.0        5
C18.5        5
154.8        4
230.3        3
153.8        3
C187         3
D01.2        3
153.2        3
V1005        3
C186         2
C189         2
230.4        1
1541         1
C181         1
153.7        1
D01.1        1
Z85048       1
Name: PATID, dtype: int64

,PATID,has_CRC,CRC_f_date,CRC_f_code
22,11e750610fb3ce06bc3a0050569ea8fb,1,2020-01-30,C18.9
25,11e75061ba921d0a94360050569ea8fb,1,2018-03-23,Z85.038
27,11e75061126eaa76a7850050569ea8fb,1,2015-05-26,153.9
28,11e75060edac71e6a7850050569ea8fb,1,2015-03-27,154.0
48,11e7506143cc4b3cab790050569ea8fb,1,2015-10-15,C18.3


In [36]:
# all CC diagnoses
df_CC = extract_history(df_dx, CC_ICD,'DX','PATID','ADMIT_DATE','DX','CRC')
df_CC.head()
df_CC.shape

df_CC.groupby('has_CRC').PATID.size()
df_CC.groupby('CRC_f_code').PATID.size().sort_values(0,ascending=False)

df_CC[df_CC.has_CRC == 1].head()

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75060feeb8046ab480050569ea8fb,0,NaT,NaN
1,11e7506106e83532bb6a0050569ea8fb,0,NaT,NaN
2,11e75060d2b34f18907a0050569ea8fb,0,NaT,NaN
3,11e75060deaca97c845d0050569ea8fb,0,NaT,NaN
4,11e75060b2681860ad370050569ea8fb,0,NaT,NaN


(35768, 4)

has_CRC
0    34554
1     1214
Name: PATID, dtype: int64

CRC_f_code
C18.9      385
153.9      162
Z85.038    135
C18.7      123
V10.05      85
C18.1       50
C18.2       47
C18.0       35
C18.4       32
C18.6       27
153.5       18
C18.3       16
153.4       14
153.3       14
D01.0       10
C18.8        9
153.1        8
153.6        7
C18.5        6
153.0        5
230.3        5
153.8        3
C187         3
153.2        3
V1005        3
Z85038       3
C186         2
C189         2
153.7        1
C181         1
Name: PATID, dtype: int64

,PATID,has_CRC,CRC_f_date,CRC_f_code
22,11e750610fb3ce06bc3a0050569ea8fb,1,2020-01-30,C18.9
25,11e75061ba921d0a94360050569ea8fb,1,2018-03-23,Z85.038
27,11e75061126eaa76a7850050569ea8fb,1,2015-05-26,153.9
48,11e7506143cc4b3cab790050569ea8fb,1,2015-10-15,C18.3
145,11e750610d398f62929b0050569ea8fb,1,2014-12-30,153.9


In [37]:
# all RC diagnoses
df_RC = extract_history(df_dx, RC_ICD,'DX','PATID','ADMIT_DATE','DX','CRC')
df_RC.head()
df_RC.shape

df_RC.groupby('has_CRC').PATID.size()
df_RC.groupby('CRC_f_code').PATID.size().sort_values(0,ascending=False)

df_RC[df_RC.has_CRC == 1].head()

,PATID,has_CRC,CRC_f_date,CRC_f_code
0,11e75060feeb8046ab480050569ea8fb,0,NaT,NaN
1,11e7506106e83532bb6a0050569ea8fb,0,NaT,NaN
2,11e75060d2b34f18907a0050569ea8fb,0,NaT,NaN
3,11e75060deaca97c845d0050569ea8fb,0,NaT,NaN
4,11e75060b2681860ad370050569ea8fb,0,NaT,NaN


(35768, 4)

has_CRC
0    35184
1      584
Name: PATID, dtype: int64

CRC_f_code
C20        248
C19        120
Z85.048     72
154.1       60
154.0       43
V10.06      29
154.8        4
D01.2        3
230.4        2
Z85048       1
D01.1        1
1541         1
Name: PATID, dtype: int64

,PATID,has_CRC,CRC_f_date,CRC_f_code
28,11e75060edac71e6a7850050569ea8fb,1,2015-03-27,154.0
264,11e7506161079cf68cac0050569ea8fb,1,2015-10-23,C19
329,11e750611f5a946694250050569ea8fb,1,2018-04-09,C20
344,11e75060b89699d2a3fa0050569ea8fb,1,2018-03-18,C20
367,11e750613216b43685f80050569ea8fb,1,2017-01-23,C20


In [38]:
df_CRC[df_CRC.has_CRC == 1] = df_CRC[df_CRC.has_CRC == 1][df_CRC.PATID.isin(list(preIMcasesCRC.PATID.unique()))]

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [39]:
df_CRC.groupby('has_CRC').PATID.size()

has_CRC
0.0    34308
1.0      909
Name: PATID, dtype: int64

In [40]:
df_CC[df_CC.has_CRC == 1] = df_CC[df_CC.has_CRC == 1][df_CC.PATID.isin(list(preIMcasesCC.PATID.unique()))]
df_CC.groupby('has_CRC').PATID.size()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


has_CRC
0.0    34554
1.0      737
Name: PATID, dtype: int64

In [41]:
df_RC[df_RC.has_CRC == 1] = df_RC[df_RC.has_CRC == 1][df_RC.PATID.isin(list(preIMcasesRC.PATID.unique()))]
df_RC.groupby('has_CRC').PATID.size()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


has_CRC
0.0    35184
1.0      311
Name: PATID, dtype: int64

In [42]:
df_CRC[df_CRC.has_CRC == 0] = df_CRC[df_CRC.has_CRC == 0][df_CRC.PATID.isin(list(preIMcontrols.PATID.unique()))]
df_CRC.groupby('has_CRC').PATID.size()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


has_CRC
0.0    22722
1.0      909
Name: PATID, dtype: int64

In [43]:
df_CC[df_CC.has_CRC == 0] = df_CC[df_CC.has_CRC == 0][df_CC.PATID.isin(list(preIMcontrols.PATID.unique()))]
df_CC.groupby('has_CRC').PATID.size()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


has_CRC
0.0    22722
1.0      737
Name: PATID, dtype: int64

In [44]:
df_RC[df_RC.has_CRC == 0] = df_RC[df_RC.has_CRC == 0][df_RC.PATID.isin(list(preIMcontrols.PATID.unique()))]
df_RC.groupby('has_CRC').PATID.size()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


has_CRC
0.0    22722
1.0      311
Name: PATID, dtype: int64

In [45]:
import datetime
def s2t(t):
    return datetime.datetime.strptime(t, "%Y-%m-%d")

def diff_days(d1, d2):
    d1 = s2t(d1)
    d2 = s2t(d2)
    return (d1-d2).days

In [46]:
df_CC_RC = pd.merge(df_CC[df_CC.has_CRC==1],df_RC[df_RC.has_CRC==1],on = 'PATID')
df_CC_RC.head()

,PATID,has_CRC_x,CRC_f_date_x,CRC_f_code_x,has_CRC_y,CRC_f_date_y,CRC_f_code_y
0,11e750611ff018b0907a0050569ea8fb,1.0,2017-10-17,C18.9,1.0,2017-10-17,C20
1,11e75060c44d8128907a0050569ea8fb,1.0,2016-10-11,C18.7,1.0,2016-10-06,C20
2,11e75061a3befc6094250050569ea8fb,1.0,2018-06-13,C18.9,1.0,2018-02-20,Z85.048
3,11e750618b7669a4ad370050569ea8fb,1.0,2014-02-25,153.6,1.0,2014-03-10,154.0
4,11e75060cba1297a85f80050569ea8fb,1.0,2016-05-25,Z85.038,1.0,2016-05-17,Z85.048


In [47]:
df_CC_RC.shape

(103, 7)

In [48]:
#df_CC_RC = df_CC_RC.astype(str)

In [49]:
#remove patients from CC and RC who had the other cancer before hand
#dup = []
#indices = []
#for i in df_CC_RC.PATID:
#    idx = df_CC_RC[df_CC_RC.PATID ==i].index
#    CCdate = df_CC_RC[df_CC_RC.PATID ==i]['CRC_f_date_x'][idx].values[0]
#    RCdate = df_CC_RC[df_CC_RC.PATID ==i]['CRC_f_date_y'][idx].values[0]
#    diff = diff_days(CCdate,RCdate)
#    if diff > 0:
#        dup.append(1)
#        indices.append(idx)
#    else:
#       dup.append(0)
#       indices.append(idx)
#dup

In [50]:
#dup = pd.DataFrame(dup)
#indices = pd.DataFrame(dup)
#df_CC_RCx = pd.concat([dup,df_CC_RC],axis =1)
#df_CC_RCx.head()

In [51]:
#RCdel = df_CC_RCx.PATID[df_CC_RCx[0]==0]
#CCdel = df_CC_RCx.PATID[df_CC_RCx[0]==1]

In [52]:
#df_CC = df_CC[~df_CC['PATID'].isin(CCdel.to_list())]
#df_CC.groupby('has_CRC').PATID.size()

In [53]:
#df_RC = df_RC[~df_RC['PATID'].isin(RCdel.to_list())]
#df_RC.groupby('has_CRC').PATID.size()

In [54]:
# first and 2nd encounter date

df_2nd_enc = _extract_2nd_record(df_dx, 'PATID', 'ADMIT_DATE', 'DX', 'AllENC')
df_first_enc = _extract_first_record(df_dx, 'PATID', 'ADMIT_DATE', 'DX', 'AllENC')

df_first_enc.head()
df_first_enc.shape
df_2nd_enc.head()
df_2nd_enc.shape

,PATID,AllENC_f_date,AllENC_f_code
324887,11e75060990fa50ea7850050569ea8fb,2012-12-15,368.13
289523,11e750609911ae628e960050569ea8fb,2012-01-25,300.00
2290345,11e75060991b041c8d0b0050569ea8fb,2012-01-31,722.4
2635203,11e7506099249b6c8d0b0050569ea8fb,2012-01-03,786.50
1586889,11e750609929551cab790050569ea8fb,2012-01-12,787.20


(35768, 3)

,PATID,AllENC_2nd_date,AllENC_2nd_code
417,11e75060feeb8046ab480050569ea8fb,2012-08-11,784.92
826,11e7506106e83532bb6a0050569ea8fb,2012-10-20,466.0
2787,11e75060d2b34f18907a0050569ea8fb,2012-01-30,585.6
3169,11e75060deaca97c845d0050569ea8fb,2012-03-27,305.1
3736,11e75060b2681860ad370050569ea8fb,2013-02-27,V72.0


(35768, 3)

In [55]:
#final list of all patients showing whether or not they have CRC, the onset date, and all 
#patient's first encounter date

df_CRC_list = [df_CRC, df_first_enc, df_2nd_enc]
df_CRC_dx = reduce(lambda left, right: pd.merge(left, right, on='PATID', how ='inner'), df_CRC_list)
df_CRC_dx.loc[df_CRC_dx.has_CRC == 1].head()
df_CRC_dx.shape

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code
17,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8
19,11e75061126eaa76a7850050569ea8fb,1.0,2015-05-26,153.9,2012-06-01,726.90,2012-07-26,719.46
20,11e75060edac71e6a7850050569ea8fb,1.0,2015-03-27,154.0,2012-01-01,585.6,2012-01-02,585.6
30,11e7506143cc4b3cab790050569ea8fb,1.0,2015-10-15,C18.3,2012-01-24,345.90,2012-02-07,338.21
87,11e750610d398f62929b0050569ea8fb,1.0,2014-12-30,153.9,2012-05-14,784.0,2012-05-17,578.1


(23631, 8)

In [56]:
#final list of all patients showing whether or not they have CC, the onset date, and all 
#patient's first encounter date

df_CC_list = [df_CC, df_first_enc, df_2nd_enc]
df_CC_dx = reduce(lambda left, right: pd.merge(left, right, on='PATID', how ='inner'), df_CC_list)
df_CC_dx.loc[df_CC_dx.has_CRC == 1].head()
df_CC_dx.shape

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code
17,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8
19,11e75061126eaa76a7850050569ea8fb,1.0,2015-05-26,153.9,2012-06-01,726.90,2012-07-26,719.46
29,11e7506143cc4b3cab790050569ea8fb,1.0,2015-10-15,C18.3,2012-01-24,345.90,2012-02-07,338.21
86,11e750610d398f62929b0050569ea8fb,1.0,2014-12-30,153.9,2012-05-14,784.0,2012-05-17,578.1
112,11e7506105c14ab8ad370050569ea8fb,1.0,2018-03-02,C18.2,2012-03-20,278.01,2013-04-09,793.80


(23459, 8)

In [57]:
#final list of all patients showing whether or not they have CRC, the onset date, and all 
#patient's first encounter date

df_RC_list = [df_RC, df_first_enc, df_2nd_enc]
df_RC_dx = reduce(lambda left, right: pd.merge(left, right, on='PATID', how ='inner'), df_RC_list)
df_RC_dx.loc[df_RC_dx.has_CRC == 1].head()
df_RC_dx.shape

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code
18,11e75060edac71e6a7850050569ea8fb,1.0,2015-03-27,154.0,2012-01-01,585.6,2012-01-02,585.6
225,11e750611ff018b0907a0050569ea8fb,1.0,2017-10-17,C20,2012-09-03,719.45,2012-09-06,401.9
250,11e75060c44d8128907a0050569ea8fb,1.0,2016-10-06,C20,2012-01-14,250.00,2012-01-27,250.03
296,11e750609da76cc8bc3a0050569ea8fb,1.0,2019-11-07,C20,2012-01-10,780.57,2012-01-16,327.23
364,11e75061a3befc6094250050569ea8fb,1.0,2018-02-20,Z85.048,2016-06-30,K64.8,2018-02-20,E87.5


(23033, 8)

# Demographic Load

In [58]:
os.chdir('/mnt/data1/songziwu/data/crc_data/')

In [59]:
df_demo = pd.read_csv('DEMOGRAPHIC.csv')
df_demo.head()
df_demo.shape

df_CRC_dx = pd.merge(df_CRC_dx, df_demo[['PATID','BIRTH_DATE','SEX','RACE','HISPANIC']],how='outer',on='PATID')
df_CRC_dx.head()
df_CRC_dx.shape

df_CC_dx = pd.merge(df_CC_dx, df_demo[['PATID','BIRTH_DATE','SEX','RACE','HISPANIC']],how='outer',on='PATID')
df_CC_dx.head()
df_CC_dx.shape

df_RC_dx = pd.merge(df_RC_dx, df_demo[['PATID','BIRTH_DATE','SEX','RACE','HISPANIC']],how='outer',on='PATID')
df_RC_dx.head()
df_RC_dx.shape

,PATID,BIRTH_DATE,BIRTH_TIME,SEX,SEXUAL_ORIENTATION,GENDER_IDENTITY,HISPANIC,RACE,BIOBANK_FLAG,PAT_PREF_LANGUAGE_SPOKEN,RAW_SEX,RAW_SEXUAL_ORIENTATION,RAW_GENDER_IDENTITY,RAW_HISPANIC,RAW_RACE,RAW_PAT_PREF_LANGUAGE_SPOKEN,UPDATED,SOURCE,ZIP_CODE,LAST_ENCOUNTERID
0,11e75060c4fbc38285f80050569ea8fb,1975-08-21,00:00,F,NI,NI,N,03,N,ENG,F,NaN,NaN,B,B,ENGLISH,2020-03-05 11:07:10.000,LNK,323401746,YvJLcrECS1asqAC4e/gf\t2019-06-27\r
1,11e75060c578f258ab480050569ea8fb,1974-09-12,04:00,M,NI,NI,Y,05,N,ENG,M,NaN,NaN,N,N,30000000|151,2020-03-05 11:07:10.000,LNK,328105977,YvJLcrECS1Ktogy5fPkX\t2019-10-04\r
2,11e75060c5a907f48d0b0050569ea8fb,1979-12-14,NaN,F,NI,NaN,N,03,N,ENG,F,NaN,NaN,B,B,ENGLISH,2020-03-05 11:07:10.000,LNK,32332,YvJLcrECS1agqwy+efEb\t2019-03-23\r
3,11e75060c649dd64a6a70050569ea8fb,1988-11-21,NaN,F,NaN,NaN,UN,UN,N,NaN,F,NaN,NaN,N,N,NaN,2020-03-05 11:07:10.000,FLM,329264079,YvJLcrECS1apowy9ffUb\t2019-05-27
4,11e75060cb80bd988e960050569ea8fb,1974-10-17,NaN,F,NI,NI,Y,05,N,NaN,F,NaN,NaN,N,N,NaN,2020-03-05 11:07:10.000,LNK,334604823,YvJLcrECS1SrrwO4e/Id\t2019-11-22\r


(35785, 20)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC
0,11e7506106e83532bb6a0050569ea8fb,0.0,NaT,NaN,2012-09-19,379.93,2012-10-20,466.0,1985-08-04,F,03,N
1,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,M,03,N
2,11e75060deaca97c845d0050569ea8fb,0.0,NaT,NaN,2012-03-14,724.2,2012-03-27,305.1,1984-02-02,F,03,N
3,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,F,03,N
4,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,F,03,N


(35785, 12)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC
0,11e7506106e83532bb6a0050569ea8fb,0.0,NaT,NaN,2012-09-19,379.93,2012-10-20,466.0,1985-08-04,F,03,N
1,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,M,03,N
2,11e75060deaca97c845d0050569ea8fb,0.0,NaT,NaN,2012-03-14,724.2,2012-03-27,305.1,1984-02-02,F,03,N
3,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,F,03,N
4,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,F,03,N


(35785, 12)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC
0,11e7506106e83532bb6a0050569ea8fb,0.0,NaT,NaN,2012-09-19,379.93,2012-10-20,466.0,1985-08-04,F,03,N
1,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,M,03,N
2,11e75060deaca97c845d0050569ea8fb,0.0,NaT,NaN,2012-03-14,724.2,2012-03-27,305.1,1984-02-02,F,03,N
3,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,F,03,N
4,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,F,03,N


(35785, 12)

In [60]:
df_CRC_dx['BIRTH_DATE'] = df_CRC_dx['BIRTH_DATE'].astype('datetime64[ns]')
df_CC_dx['BIRTH_DATE'] = df_CC_dx['BIRTH_DATE'].astype('datetime64[ns]')
df_RC_dx['BIRTH_DATE'] = df_RC_dx['BIRTH_DATE'].astype('datetime64[ns]')

In [61]:
df_CRC_dx.groupby('has_CRC').size()
df_CC_dx.groupby('has_CRC').size()
df_RC_dx.groupby('has_CRC').size()

has_CRC
0.0    22722
1.0      909
dtype: int64

has_CRC
0.0    22722
1.0      737
dtype: int64

has_CRC
0.0    22722
1.0      311
dtype: int64

In [62]:
#if first encounter is before birthdate for some reason for diagnosis like Crohns move to birthdate 

log = df_CRC_dx.AllENC_f_date < df_CRC_dx.BIRTH_DATE
df_CRC_dx[log].shape
print(df_CRC_dx[log])

df_CRC_dx.AllENC_f_date[log] = df_CRC_dx.AllENC_2nd_date[log]
print(df_CRC_dx[log])

log = df_CC_dx.AllENC_f_date < df_CC_dx.BIRTH_DATE
df_CC_dx[log].shape
print(df_CC_dx[log])

df_CC_dx.AllENC_f_date[log] = df_CC_dx.AllENC_2nd_date[log]
print(df_CC_dx[log])

log = df_RC_dx.AllENC_f_date < df_RC_dx.BIRTH_DATE
df_RC_dx[log].shape
print(df_RC_dx[log])

df_RC_dx.AllENC_f_date[log] = df_RC_dx.AllENC_2nd_date[log]
print(df_RC_dx[log])

df_CRC_dx_match = df_CRC_dx
df_CC_dx_match = df_CC_dx
df_RC_dx_match = df_RC_dx

(1, 12)

                                 PATID  has_CRC CRC_f_date CRC_f_code  \
7318  11e75066cf3b196eab790050569ea8fb      0.0        NaT        NaN   

     AllENC_f_date AllENC_f_code AllENC_2nd_date AllENC_2nd_code BIRTH_DATE  \
7318    1900-01-01        K50.90      2013-01-30          728.85 2000-12-01   

     SEX RACE HISPANIC  
7318   F   05        Y  


/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


                                 PATID  has_CRC CRC_f_date CRC_f_code  \
7318  11e75066cf3b196eab790050569ea8fb      0.0        NaT        NaN   

     AllENC_f_date AllENC_f_code AllENC_2nd_date AllENC_2nd_code BIRTH_DATE  \
7318    2013-01-30        K50.90      2013-01-30          728.85 2000-12-01   

     SEX RACE HISPANIC  
7318   F   05        Y  


(1, 12)

...

                                 PATID  has_CRC CRC_f_date CRC_f_code  \
7263  11e75066cf3b196eab790050569ea8fb      0.0        NaT        NaN   

     AllENC_f_date AllENC_f_code AllENC_2nd_date AllENC_2nd_code BIRTH_DATE  \
7263    2013-01-30        K50.90      2013-01-30          728.85 2000-12-01   

     SEX RACE HISPANIC  
7263   F   05        Y  


(1, 12)

                                 PATID  has_CRC CRC_f_date CRC_f_code  \
7130  11e75066cf3b196eab790050569ea8fb      0.0        NaT        NaN   

     AllENC_f_date AllENC_f_code AllENC_2nd_date AllENC_2nd_code BIRTH_DATE  \
7130    1900-01-01        K50.90      2013-01-30          728.85 2000-12-01   

     SEX RACE HISPANIC  
7130   F   05        Y  


/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                                 PATID  has_CRC CRC_f_date CRC_f_code  \
7130  11e75066cf3b196eab790050569ea8fb      0.0        NaT        NaN   

     AllENC_f_date AllENC_f_code AllENC_2nd_date AllENC_2nd_code BIRTH_DATE  \
7130    2013-01-30        K50.90      2013-01-30          728.85 2000-12-01   

     SEX RACE HISPANIC  
7130   F   05        Y  


# Matching

Incident Matching
    1. index date = first encounter date at which patient was diagnosed with AD/RD
    2. reference date of control = the encounter date within 30 days of the index date of the case
    3. first date = first encounter date of the patients
    4. watching period date = first encounter date that allows more than the time specified by the prediction window 
    5. observation window = all days from the first encounter date to the watching period date
    
Exclusion Criteria
    1. observation criteria < 2 years
    
Incident Matching Criteria
    1. Match to age within +- 2.5 years
    2. Having an encounter within 30-day interval of the case's onset date
    3. Two AV encounters that are at least two years apart before index date
    4. Age at index date >=18

In [63]:
def match_case_control(df, df_long, longdatevar, patidvar, casevar, bdatevar, fencdatevar, 
                       bdatediff, encdatediff, caseindexdatevar, matchratio, datatimespan):
    
    case_id_list = list(df[df[casevar] == 1][patidvar])

    _df_control_temp = pd.DataFrame([])
    _matched_control_id_set = set()
    for caseid in case_id_list:
        # extract matching criteria values for case
        case_bdate = df[df[patidvar] == caseid][bdatevar].values[0]
        case_indexdate = df[df[patidvar] == caseid][caseindexdatevar].values[0]
    
        # matching criteria
        in_control_pool = (df[casevar] == 0)
        match_bdate = (abs(df[bdatevar] - case_bdate) / np.timedelta64(1, 'D') <= bdatediff)       
        two_years = (abs(df[fencdatevar] - case_indexdate) /np.timedelta64(1, 'D') >= datatimespan)
        
        # non-replacing sampling
        not_in_matched_set = (~df[patidvar].isin(_matched_control_id_set))
        
        # match control step 1 of 2
        _df_control_per_temp = df[in_control_pool & match_bdate & not_in_matched_set & two_years]
    
        # match index date for control, and create reference date, step 2 of 2
        ## select df_long subset for candidate control only
        _df_control_long_temp_1 = df_long[df_long[patidvar].isin(list(_df_control_per_temp[patidvar]))]
        
        
        ## create new column for the gap between reference date and index date of the case
        _df_control_long_temp_1['ref_index_gap'] = abs((_df_control_long_temp_1[longdatevar] - case_indexdate) / np.timedelta64(1, 'D'))
        
        ## locate the reference date with smallest gap
        _df_control_long_temp_2 = _df_control_long_temp_1.loc[_df_control_long_temp_1.groupby(patidvar)['ref_index_gap'].idxmin(), [patidvar,longdatevar,'ref_index_gap']]
        
        ## remove the control from the candidate list if the smallest gap larger than 'encdatediff'
        _df_control_long_temp_2 = _df_control_long_temp_2[_df_control_long_temp_2['ref_index_gap'] <= encdatediff]
        
        ## rename for the reference date
        _df_control_long_temp_2.rename(columns={longdatevar:'ref_index_date'}, inplace=True)
        
        ## merge the patient level data back, all matching criteria met!
        _df_control_per_temp = pd.merge(_df_control_per_temp, _df_control_long_temp_2, on=patidvar, how='inner')
        
        # restrict match ratio by selecting the closest birth date to the case
        _df_control_per_temp['bdate_gap_w_case'] = abs((_df_control_per_temp[bdatevar] - case_bdate) / np.timedelta64(1, 'D'))
    
        if _df_control_per_temp.shape[0] > matchratio:
            _df_control_per_temp = _df_control_per_temp.nsmallest(matchratio, 'bdate_gap_w_case')
            
        # record which case does these controal matched to
        _df_control_per_temp['matched_case_id'] = caseid

        # append the results (control list)
        _df_control_temp = _df_control_temp.append(_df_control_per_temp)

        # non-replacing sampling, put the matched control into the matched pool
        _matched_control_id_set.update(_df_control_per_temp[patidvar].values)

    return _df_control_temp

    
df_matched_control_CRC = match_case_control(df_CRC_dx_match, df_dx,'ADMIT_DATE','PATID','has_CRC','BIRTH_DATE','AllENC_f_date',
                                        912,30,'CRC_f_date',10,730)

df_matched_control_CC = match_case_control(df_CC_dx_match, df_dx,'ADMIT_DATE','PATID','has_CRC','BIRTH_DATE','AllENC_f_date',
                                        912,30,'CRC_f_date',10,730)

df_matched_control_RC = match_case_control(df_RC_dx_match, df_dx,'ADMIT_DATE','PATID','has_CRC','BIRTH_DATE','AllENC_f_date',
                                        912,30,'CRC_f_date',10,730)

df_matched_control_CRC.head()
df_matched_control_CRC.shape

df_matched_control_CC.head()
df_matched_control_CC.shape

df_matched_control_RC.head()
df_matched_control_RC.shape

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,ref_index_gap,bdate_gap_w_case,matched_case_id
72,11e7506a0165fec4845d0050569ea8fb,0.0,NaT,NaN,2012-03-14,556.9,2012-05-14,556.9,1981-05-22,M,05,N,2020-01-21,9.0,3.0,11e750610fb3ce06bc3a0050569ea8fb
112,11e7506e230386cea3fa0050569ea8fb,0.0,NaT,NaN,2014-04-11,737.30,2014-04-23,373.13,1981-06-01,F,05,N,2020-01-27,3.0,7.0,11e750610fb3ce06bc3a0050569ea8fb
106,11e8217827601936acad0050569ea8fb,0.0,NaT,NaN,2012-05-03,V85.36,2012-05-31,V53.32,1981-05-16,M,OT,Y,2020-01-02,28.0,9.0,11e750610fb3ce06bc3a0050569ea8fb
121,11e750befbd8489a9c190050569ea8fb,0.0,NaT,NaN,2012-01-04,696.1,2012-01-26,110.4,1981-05-06,M,05,N,2020-01-09,21.0,19.0,11e750610fb3ce06bc3a0050569ea8fb
123,11e82178303fd6aeb5ac0050569ea8fb,0.0,NaT,NaN,2012-05-01,V58.61,2012-05-28,314.00,1981-06-13,F,05,Y,2020-01-14,16.0,19.0,11e750610fb3ce06bc3a0050569ea8fb


(8665, 16)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,ref_index_gap,bdate_gap_w_case,matched_case_id
72,11e7506a0165fec4845d0050569ea8fb,0.0,NaT,NaN,2012-03-14,556.9,2012-05-14,556.9,1981-05-22,M,05,N,2020-01-21,9.0,3.0,11e750610fb3ce06bc3a0050569ea8fb
112,11e7506e230386cea3fa0050569ea8fb,0.0,NaT,NaN,2014-04-11,737.30,2014-04-23,373.13,1981-06-01,F,05,N,2020-01-27,3.0,7.0,11e750610fb3ce06bc3a0050569ea8fb
106,11e8217827601936acad0050569ea8fb,0.0,NaT,NaN,2012-05-03,V85.36,2012-05-31,V53.32,1981-05-16,M,OT,Y,2020-01-02,28.0,9.0,11e750610fb3ce06bc3a0050569ea8fb
121,11e750befbd8489a9c190050569ea8fb,0.0,NaT,NaN,2012-01-04,696.1,2012-01-26,110.4,1981-05-06,M,05,N,2020-01-09,21.0,19.0,11e750610fb3ce06bc3a0050569ea8fb
123,11e82178303fd6aeb5ac0050569ea8fb,0.0,NaT,NaN,2012-05-01,V58.61,2012-05-28,314.00,1981-06-13,F,05,Y,2020-01-14,16.0,19.0,11e750610fb3ce06bc3a0050569ea8fb


(7004, 16)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,ref_index_gap,bdate_gap_w_case,matched_case_id
76,11e7506103f84ddaab790050569ea8fb,0.0,NaT,NaN,2012-01-06,996.52,2012-01-10,996.52,1966-02-28,F,05,N,2015-03-25,2.0,1.0,11e75060edac71e6a7850050569ea8fb
228,11e750621ad8f440ba790050569ea8fb,0.0,NaT,NaN,2012-03-27,296.90,2012-04-10,272.0,1966-02-28,M,05,N,2015-03-27,0.0,1.0,11e75060edac71e6a7850050569ea8fb
106,11e75060f96859e6a7850050569ea8fb,0.0,NaT,NaN,2012-05-22,V73.88,2012-06-01,295.30,1966-02-25,F,03,N,2015-03-13,14.0,2.0,11e75060edac71e6a7850050569ea8fb
461,11e750648424bcb68d0b0050569ea8fb,0.0,NaT,NaN,2012-10-23,355.9,2012-10-24,786.59,1966-03-01,F,05,N,2015-03-26,1.0,2.0,11e75060edac71e6a7850050569ea8fb
670,11e75068cb6e64f688ed0050569ea8fb,0.0,NaT,NaN,2012-01-25,V72.83,2012-02-22,715.97,1966-02-25,M,05,N,2015-03-24,3.0,2.0,11e75060edac71e6a7850050569ea8fb


(3053, 16)

In [64]:
df_matched_control_CRC.matched_case_id.nunique()
df_matched_control_CC.matched_case_id.nunique()
df_matched_control_RC.matched_case_id.nunique()

871

704

308

In [65]:
df_matched_control_CRC.PATID.nunique()
df_matched_control_CC.PATID.nunique()
df_matched_control_RC.PATID.nunique()

8665

7004

3053

In [66]:
df_matched_case_control_CRC = df_CRC_dx[(df_CRC_dx.PATID.isin(set(df_matched_control_CRC.PATID))) 
                                    | (df_CRC_dx.PATID.isin(set(df_matched_control_CRC.matched_case_id)))]

df_matched_case_control_CRC = df_matched_case_control_CRC.merge(df_matched_control_CRC[['PATID','ref_index_date']], right_on = 'PATID', left_on = 'PATID', how = 'left')
df_matched_case_control_CRC.head()


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,M,03,N,2014-07-30
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,F,03,N,2014-08-05
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,F,05,N,2018-12-09
3,11e75060ded868c8bb6a0050569ea8fb,0.0,NaT,NaN,2012-01-20,338.19,2012-04-23,616.10,1985-02-11,F,03,N,2019-11-21
4,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,F,03,N,2015-12-02


In [67]:
df_matched_case_control_CC = df_CC_dx[(df_CC_dx.PATID.isin(set(df_matched_control_CC.PATID))) 
                                    | (df_CC_dx.PATID.isin(set(df_matched_control_CC.matched_case_id)))]

df_matched_case_control_CC = df_matched_case_control_CC.merge(df_matched_control_CC[['PATID','ref_index_date']], right_on = 'PATID', left_on = 'PATID', how = 'left')
df_matched_case_control_CC.head()


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,M,03,N,2015-04-17
1,11e75060ded868c8bb6a0050569ea8fb,0.0,NaT,NaN,2012-01-20,338.19,2012-04-23,616.10,1985-02-11,F,03,N,2018-08-15
2,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,F,03,N,2015-12-02
3,11e75060f38d5454bf840050569ea8fb,0.0,NaT,NaN,2012-09-21,319,2012-09-25,V01.6,1986-05-31,F,03,N,2017-11-30
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,F,03,N,NaT


In [68]:
df_matched_case_control_RC = df_RC_dx[(df_RC_dx.PATID.isin(set(df_matched_control_RC.PATID))) 
                                    | (df_RC_dx.PATID.isin(set(df_matched_control_RC.matched_case_id)))]

df_matched_case_control_RC = df_matched_case_control_RC.merge(df_matched_control_RC[['PATID','ref_index_date']], right_on = 'PATID', left_on = 'PATID', how = 'left')
df_matched_case_control_RC.head()


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date
0,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,F,03,N,2019-05-15
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,F,03,N,2014-08-05
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,F,05,N,2018-12-09
3,11e75060ded868c8bb6a0050569ea8fb,0.0,NaT,NaN,2012-01-20,338.19,2012-04-23,616.10,1985-02-11,F,03,N,2019-07-08
4,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,F,05,N,2014-02-28


# PSMatching

In [69]:
def fun_race3(race):
    if race in ['01','04','06','OT']:
        return 'Other'
    elif race == '02':
        return 'Asian'
    elif race == '03':
        return 'Black'
    elif race == '05':
        return 'White'
    else:
        return 'Unknown'

def fun_mpec(mpec):
    if mpec >= 0.6:
        return 1
    else:
        return 0
    
def fun_ethnicity(ethnicity):
    if ethnicity == 'Y':
        return 'Hispanic'
    elif ethnicity == 'N':
        return 'Non-Hispanic'
    else:
        return 'Unknown'

def fun_race_ethnicity(race, ethnicity):
    if ethnicity == 'Y':
        return 'Hispanic'
    elif ethnicity == 'N' and race == 'White':
        return 'NHW'
    elif ethnicity == 'N' and race == 'Black':
        return 'NHB'
    elif ethnicity == 'N' and race == 'Other':
        return 'NHOther'
    else:
        return 'Unknown'

# encode SEX

def label_encoding(df, fea):
    le = preprocessing.LabelEncoder()
    le.fit(df[fea])
    df[fea] = le.transform(df[fea])
    return df

def process_psm_feature(df):
    df['race_c3'] = df.apply(lambda x: fun_race3(x['RACE']), axis=1)
    df['ethnicity'] = df.apply(lambda x: fun_ethnicity(x['HISPANIC']), axis=1)
    df['race_ethnicity'] = df.apply(lambda x: fun_race_ethnicity(x['race_c3'], x['HISPANIC']), axis=1)
    
    df = label_encoding(df, 'SEX')
    df = label_encoding(df, 'race_ethnicity')
    
    df['BIRTH_YEAR'] = pd.DatetimeIndex(df['BIRTH_DATE']).year
    df['has_CRC'] = df['has_CRC'].fillna(value=0)
    return df


In [70]:
df_CRC_psmatch = process_psm_feature(df_matched_case_control_CRC)
df_CC_psmatch = process_psm_feature(df_matched_case_control_CC)
df_RC_psmatch = process_psm_feature(df_matched_case_control_RC)

In [71]:
df_CC_psmatch.head()

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2015-04-17,Black,Non-Hispanic,1,1974
1,11e75060ded868c8bb6a0050569ea8fb,0.0,NaT,NaN,2012-01-20,338.19,2012-04-23,616.10,1985-02-11,0,03,N,2018-08-15,Black,Non-Hispanic,1,1985
2,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982
3,11e75060f38d5454bf840050569ea8fb,0.0,NaT,NaN,2012-09-21,319,2012-09-25,V01.6,1986-05-31,0,03,N,2017-11-30,Black,Non-Hispanic,1,1986
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

predictors = ['BIRTH_YEAR', 'SEX','race_ethnicity']

#CRC
propensity = LogisticRegression()
propensity = propensity.fit(df_CRC_psmatch[predictors], df_CRC_psmatch.has_CRC)

df_CRC_psmatch['pscore'] = propensity.predict_proba(df_CRC_psmatch[predictors])[:,1]

#CC
propensity = LogisticRegression()
propensity = propensity.fit(df_CC_psmatch[predictors], df_CC_psmatch.has_CRC)

df_CC_psmatch['pscore'] = propensity.predict_proba(df_CC_psmatch[predictors])[:,1]

#RC
propensity = LogisticRegression()
propensity = propensity.fit(df_RC_psmatch[predictors], df_RC_psmatch.has_CRC)

df_RC_psmatch['pscore'] = propensity.predict_proba(df_RC_psmatch[predictors])[:,1]

/home/chongdang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/chongdang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/chongdang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [74]:
def set_caliper(caliper_scale, propensity, caliper=0.05):
    # Check inputs
    if caliper_scale == None:
        caliper = 0
    if not(0<=caliper<1):
        if caliper_scale == "propensity" and caliper>1:
            raise ValueError('Caliper for "propensity" method must be between 0 and 1')
        elif caliper<0:
            raise ValueError('Caliper cannot be negative')

    # Transform the propensity scores and caliper when caliper_scale is "logit" or None
    if caliper_scale == "logit":
        propensity = np.log(propensity/(1-propensity))
        caliper = caliper*np.std(propensity)
    
#     print(caliper)
    return caliper

def match_by_neighbor(data, ratio = 5): # 1 to ratio matching
    
    # make sure index date is datetime format

    controls = data[data.has_CRC == 0]
    cases = data[data.has_CRC == 1]
    
    # a few paper suggested to use 0.2 of the propensity score (on the logit scale) as the caliper
    # but in reality, the smaller the caliper, the better the matching 
    # So choose a small number if you have sufficient controls 
    caliper = set_caliper('logit', data.pscore, caliper=0.01)
    
    neigh = NearestNeighbors(radius=caliper, algorithm='ball_tree', n_jobs=1)
    neigh.fit(controls[['pscore']])

    ignore_list = set()
    matched_controls = pd.DataFrame()
    under_matched = set()
    unmatched = set()
    sample_size=pd.DataFrame()

    i = 1
    total_cases = cases.shape[0]
    start = timeit.default_timer()
    
    #loop through each case
    for index, case in cases.iterrows():
        
        # case index date
        case_indexdate = cases[cases.PATID == case['PATID']].CRC_f_date.values[0]
        
        # current case's pscore
        pscore = case.pscore
        
        # find all controls with pscore within the caliper distance
        distances, indices = neigh.radius_neighbors([[pscore]])
        
        sample = controls.iloc[indices[0]]
        
        # pick out those that have NOT been used
        sample = sample[~sample['PATID'].isin(ignore_list)].copy()
        
        ## verify index date for control
        sample['index_date_gap'] = abs(sample.ref_index_date - case_indexdate) / np.timedelta64(1, 'D')
        sample = sample[sample.index_date_gap <= 180].sort_values(by=['PATID', 'index_date_gap'])
        
        # rank the samples by their distances to the case's pscore
        sample['d'] = abs(sample['pscore']-pscore)
        sample.sort_values(by='d', ascending=True, inplace=True)
        
        # picked the closest "ratio"
        sample = sample.head(ratio).copy().reset_index(drop=True)
        
        if (sample.shape[0] < ratio and sample.shape[0] != 0):
            under_matched.add(case['PATID'])
            
        if (sample.shape[0] == 0):
            unmatched.add(case['PATID'])
            
        # exclude the selected sample from the matching pool (i.e., without replacement)
        ignore_list.update(sample['PATID'])
        
        sample['matched_case'] = case['PATID']
        sample['matched_case_index_date'] = case_indexdate
        
        matched_controls = matched_controls.append(sample, ignore_index=True)
        
        # track progress
        clear_output(wait=True)
        stop = timeit.default_timer()
        
        print("Current progress:", np.round(i/total_cases * 100, 2), "%")
        print("Current run time:", np.round((stop - start) / 60, 2), "min")
        
        i = i+1
        
    return under_matched, unmatched, matched_controls.reset_index(drop=True)

In [75]:
under_matched_CRC, unmatched_CRC, controls_CRC = match_by_neighbor(df_CRC_psmatch)

Current progress: 100.0 %
Current run time: 0.42 min


In [76]:
under_matched_CC, unmatched_CC, controls_CC = match_by_neighbor(df_CC_psmatch)

Current progress: 100.0 %
Current run time: 0.31 min


In [77]:
under_matched_RC, unmatched_RC, controls_RC = match_by_neighbor(df_RC_psmatch)

Current progress: 100.0 %
Current run time: 0.14 min


In [78]:
df_matched_case_control_CRC = df_CRC_psmatch[(df_CRC_psmatch.PATID.isin(set(controls_CRC.PATID))) 
                                    | (df_CRC_psmatch.PATID.isin(set(controls_CRC.matched_case)))]

In [79]:
df_matched_case_control_CC = df_CC_psmatch[(df_CC_psmatch.PATID.isin(set(controls_CC.PATID))) 
                                    | (df_CC_psmatch.PATID.isin(set(controls_CC.matched_case)))]

In [80]:
df_matched_case_control_RC = df_RC_psmatch[(df_RC_psmatch.PATID.isin(set(controls_RC.PATID))) 
                                    | (df_RC_psmatch.PATID.isin(set(controls_RC.matched_case)))]

In [81]:
# df_matched_case_control_CRC = pd.merge(df_matched_case_control_CRC, controls_CRC[['PATID','ref_index_date']],on=['PATID'],how='outer')
# df_matched_case_control_CRC[df_matched_case_control_CRC.has_CRC == 1].head()
# df_matched_case_control_CRC.shape

In [82]:
# df_matched_case_control_CC = pd.merge(df_matched_case_control_CC,controls_CC[['PATID','ref_index_date']],on=['PATID'],how='outer')
# df_matched_case_control_CC[df_matched_case_control_CC.has_CRC == 1].head()
# df_matched_case_control_CC.shape

In [83]:
# df_matched_case_control_RC = pd.merge(df_matched_case_control_RC,controls_RC[['PATID','ref_index_date']],on=['PATID'],how='outer')
# df_matched_case_control_RC[df_matched_case_control_RC.has_CRC == 1].head()
# df_matched_case_control_RC.shape

In [84]:
df_matched_case_control_CRC.head()

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2014-07-30,Black,Non-Hispanic,1,1974,0.091605
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.091605
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546
4,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090834
5,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.090355


In [85]:
# define index date
def fun_index_date(status, case_date, control_date):
    if status == 1:
        return case_date
    elif status == 0:
        return control_date
    
df_matched_case_control_CRC['index_CRC'] = df_matched_case_control_CRC.apply(lambda x: fun_index_date(
    x['has_CRC'],x['CRC_f_date'],x['ref_index_date']), axis=1)

df_matched_case_control_CRC[df_matched_case_control_CRC.has_CRC == 1].head()
df_matched_case_control_CRC[df_matched_case_control_CRC.has_CRC == 0].head()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC
8,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090930,2020-01-30
9,11e75061126eaa76a7850050569ea8fb,1.0,2015-05-26,153.9,2012-06-01,726.90,2012-07-26,719.46,1966-03-26,0,05,N,NaT,White,Non-Hispanic,3,1966,0.092381,2015-05-26
10,11e75060edac71e6a7850050569ea8fb,1.0,2015-03-27,154.0,2012-01-01,585.6,2012-01-02,585.6,1966-02-27,0,03,N,NaT,Black,Non-Hispanic,1,1966,0.092381,2015-03-27
16,11e7506143cc4b3cab790050569ea8fb,1.0,2015-10-15,C18.3,2012-01-24,345.90,2012-02-07,338.21,1968-10-06,0,05,N,NaT,White,Non-Hispanic,3,1968,0.092186,2015-10-15
49,11e750610d398f62929b0050569ea8fb,1.0,2014-12-30,153.9,2012-05-14,784.0,2012-05-17,578.1,1966-02-14,0,05,N,NaT,White,Non-Hispanic,3,1966,0.092381,2014-12-30


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2014-07-30,Black,Non-Hispanic,1,1974,0.091605,2014-07-30
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.091605,2014-08-05
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546,2018-12-09
4,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090834,2015-12-02
5,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.090355,2014-02-28


In [86]:
df_matched_case_control_CC['index_CRC'] = df_matched_case_control_CC.apply(lambda x: fun_index_date(
    x['has_CRC'],x['CRC_f_date'],x['ref_index_date']), axis=1)

df_matched_case_control_CC[df_matched_case_control_CC.has_CRC == 1].head()
df_matched_case_control_CC[df_matched_case_control_CC.has_CRC == 0].head()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090940,2020-01-30
5,11e75061126eaa76a7850050569ea8fb,1.0,2015-05-26,153.9,2012-06-01,726.90,2012-07-26,719.46,1966-03-26,0,05,N,NaT,White,Non-Hispanic,3,1966,0.092391,2015-05-26
11,11e7506143cc4b3cab790050569ea8fb,1.0,2015-10-15,C18.3,2012-01-24,345.90,2012-02-07,338.21,1968-10-06,0,05,N,NaT,White,Non-Hispanic,3,1968,0.092196,2015-10-15
37,11e750610d398f62929b0050569ea8fb,1.0,2014-12-30,153.9,2012-05-14,784.0,2012-05-17,578.1,1966-02-14,0,05,N,NaT,White,Non-Hispanic,3,1966,0.092391,2014-12-30
50,11e7506105c14ab8ad370050569ea8fb,1.0,2018-03-02,C18.2,2012-03-20,278.01,2013-04-09,793.80,1981-12-30,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090940,2018-03-02


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2015-04-17,Black,Non-Hispanic,1,1974,0.091615,2015-04-17
2,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090844,2015-12-02
7,11e75060e4aded54907a0050569ea8fb,0.0,NaT,NaN,2012-01-10,530.81,2012-01-13,465.9,1986-03-13,0,05,N,2016-10-04,White,Non-Hispanic,3,1986,0.090460,2016-10-04
8,11e75060b4f00d4abb6a0050569ea8fb,0.0,NaT,NaN,2012-04-16,616.10,2012-11-25,787.02,1985-10-04,0,03,N,2015-12-03,Black,Non-Hispanic,1,1985,0.090556,2015-12-03
12,11e75060af8bbd5eab480050569ea8fb,0.0,NaT,NaN,2012-01-06,786.50,2012-01-27,278.00,1973-12-03,0,03,N,2019-04-02,Black,Non-Hispanic,1,1973,0.091711,2019-04-02


In [87]:
df_matched_case_control_RC['index_CRC'] = df_matched_case_control_RC.apply(lambda x: fun_index_date(
    x['has_CRC'],x['CRC_f_date'],x['ref_index_date']), axis=1)

df_matched_case_control_RC[df_matched_case_control_RC.has_CRC == 1].head()
df_matched_case_control_RC[df_matched_case_control_RC.has_CRC == 0].head()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC
6,11e75060edac71e6a7850050569ea8fb,1.0,2015-03-27,154.0,2012-01-01,585.6,2012-01-02,585.6,1966-02-27,0,03,N,NaT,Black,Non-Hispanic,1,1966,0.086017,2015-03-27
50,11e750611ff018b0907a0050569ea8fb,1.0,2017-10-17,C20,2012-09-03,719.45,2012-09-06,401.9,1970-08-26,0,05,N,NaT,White,Non-Hispanic,3,1970,0.097830,2017-10-17
56,11e75060c44d8128907a0050569ea8fb,1.0,2016-10-06,C20,2012-01-14,250.00,2012-01-27,250.03,1985-01-03,0,05,N,NaT,White,Non-Hispanic,3,1985,0.096201,2016-10-06
64,11e750609da76cc8bc3a0050569ea8fb,1.0,2019-11-07,C20,2012-01-10,780.57,2012-01-16,327.23,1976-04-20,0,03,N,NaT,Black,Non-Hispanic,1,1976,0.085048,2019-11-07
79,11e75061a3befc6094250050569ea8fb,1.0,2018-02-20,Z85.048,2016-06-30,K64.8,2018-02-20,E87.5,1991-03-07,0,03,N,NaT,Black,Non-Hispanic,1,1991,0.083613,2018-02-20


,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC
0,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,0,03,N,2019-05-15,Black,Non-Hispanic,1,1979,0.084759,2019-05-15
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.085241,2014-08-05
4,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.095986,2014-02-28
7,11e750612fc38484ab480050569ea8fb,0.0,NaT,NaN,2012-02-28,356.0,2012-03-12,359.21,1985-07-21,1,05,N,2018-12-18,White,Non-Hispanic,3,1985,0.098430,2018-12-18
10,11e75060c2527d1088ed0050569ea8fb,0.0,NaT,NaN,2012-03-10,250.00,2012-03-16,250.00,1975-04-21,0,03,N,2017-11-14,Black,Non-Hispanic,1,1975,0.085145,2017-11-14


In [88]:
df_matched_case_control_CRC.groupby('has_CRC').size()
df_matched_case_control_CC.groupby('has_CRC').size()
df_matched_case_control_RC.groupby('has_CRC').size()

has_CRC
0.0    4342
1.0     870
dtype: int64

has_CRC
0.0    3508
1.0     702
dtype: int64

has_CRC
0.0    1496
1.0     306
dtype: int64

In [89]:
# find amount of data in years between index date and first encounter
df_matched_case_control_CRC['FENC2idx_years'] = (df_matched_case_control_CRC['index_CRC'] 
                                             - df_matched_case_control_CRC['AllENC_f_date']) / np.timedelta64(1, 'D') / 365 
df_matched_case_control_CRC.FENC2idx_years.describe()
#df_matched_case_control_CRC.loc[df_matched_case_control_CRC.FENC2idx_years == df_matched_case_control_CRC.FENC2idx_years.max()]
#df_matched_case_control_CRC.loc[df_matched_case_control_CRC.FENC2idx_years == df_matched_case_control_CRC.FENC2idx_years.min()]

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count    5212.000000
mean        4.298065
std         1.680037
min         0.000000
25%         2.868493
50%         4.101370
75%         5.619178
max         8.073973
Name: FENC2idx_years, dtype: float64

In [90]:
df_matched_case_control_CC['FENC2idx_years'] = (df_matched_case_control_CC['index_CRC'] 
                                             - df_matched_case_control_CC['AllENC_f_date']) / np.timedelta64(1, 'D') / 365 
df_matched_case_control_CC.FENC2idx_years.describe()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    4210.000000
mean        4.355473
std         1.684435
min         0.000000
25%         2.972603
50%         4.149315
75%         5.682192
max         8.054795
Name: FENC2idx_years, dtype: float64

In [91]:
df_matched_case_control_RC['FENC2idx_years'] = (df_matched_case_control_RC['index_CRC'] 
                                             - df_matched_case_control_RC['AllENC_f_date']) / np.timedelta64(1, 'D') / 365 
df_matched_case_control_RC.FENC2idx_years.describe()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    1802.000000
mean        4.488223
std         1.738797
min         0.000000
25%         2.918493
50%         4.412329
75%         6.039726
max         9.778082
Name: FENC2idx_years, dtype: float64

In [92]:
# compute age of each patient at index using amount of data computed above
df_matched_case_control_CRC['age_index'] = (df_matched_case_control_CRC.index_CRC - df_matched_case_control_CRC.BIRTH_DATE).astype('<m8[Y]')
df_matched_case_control_CRC.age_index.describe()
df_matched_case_control_CRC.head()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    5212.000000
mean       39.292018
std         7.901931
min        17.000000
25%        34.000000
50%        41.000000
75%        46.000000
max        52.000000
Name: age_index, dtype: float64

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2014-07-30,Black,Non-Hispanic,1,1974,0.091605,2014-07-30,2.575342,39.0
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.091605,2014-08-05,2.421918,39.0
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546,2018-12-09,6.013699,33.0
4,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090834,2015-12-02,3.452055,33.0
5,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.090355,2014-02-28,2.153425,26.0


In [93]:
df_matched_case_control_CC['age_index'] = (df_matched_case_control_CC.index_CRC - df_matched_case_control_CC.BIRTH_DATE).astype('<m8[Y]')
df_matched_case_control_CC.age_index.describe()
df_matched_case_control_CC.head()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


count    4210.000000
mean       39.173159
std         7.892946
min        17.000000
25%        34.000000
50%        41.000000
75%        46.000000
max        51.000000
Name: age_index, dtype: float64

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2015-04-17,Black,Non-Hispanic,1,1974,0.091615,2015-04-17,3.290411,40.0
2,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090844,2015-12-02,3.452055,33.0
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090940,2020-01-30,8.035616,38.0
5,11e75061126eaa76a7850050569ea8fb,1.0,2015-05-26,153.9,2012-06-01,726.90,2012-07-26,719.46,1966-03-26,0,05,N,NaT,White,Non-Hispanic,3,1966,0.092391,2015-05-26,2.983562,49.0
7,11e75060e4aded54907a0050569ea8fb,0.0,NaT,NaN,2012-01-10,530.81,2012-01-13,465.9,1986-03-13,0,05,N,2016-10-04,White,Non-Hispanic,3,1986,0.090460,2016-10-04,4.736986,30.0


In [94]:
df_matched_case_control_RC['age_index'] = (df_matched_case_control_RC.index_CRC - df_matched_case_control_RC.BIRTH_DATE).astype('<m8[Y]')
df_matched_case_control_RC.age_index.describe()
df_matched_case_control_RC.head()

/home/chongdang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


count    1802.000000
mean       40.452275
std         7.242618
min        18.000000
25%        35.000000
50%        42.000000
75%        46.000000
max        52.000000
Name: age_index, dtype: float64

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,0,03,N,2019-05-15,Black,Non-Hispanic,1,1979,0.084759,2019-05-15,6.890411,40.0
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.085241,2014-08-05,2.421918,39.0
4,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.095986,2014-02-28,2.153425,26.0
6,11e75060edac71e6a7850050569ea8fb,1.0,2015-03-27,154.0,2012-01-01,585.6,2012-01-02,585.6,1966-02-27,0,03,N,NaT,Black,Non-Hispanic,1,1966,0.086017,2015-03-27,3.235616,49.0
7,11e750612fc38484ab480050569ea8fb,0.0,NaT,NaN,2012-02-28,356.0,2012-03-12,359.21,1985-07-21,1,05,N,2018-12-18,White,Non-Hispanic,3,1985,0.098430,2018-12-18,6.808219,33.0


In [95]:
# look at patients with less than 2 years of data
df_matched_case_control_CRC[(df_matched_case_control_CRC.FENC2idx_years < 2)].shape
df_matched_case_control_CC[(df_matched_case_control_CC.FENC2idx_years < 2)].shape
df_matched_case_control_RC[(df_matched_case_control_RC.FENC2idx_years < 2)].shape

(133, 21)

(104, 21)

(51, 21)

In [96]:
df_matched_case_control_CRC.shape
df_matched_case_control_CRC.groupby('has_CRC').size()
df_matched_case_control_CRC.head()

(5212, 21)

has_CRC
0.0    4342
1.0     870
dtype: int64

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2014-07-30,Black,Non-Hispanic,1,1974,0.091605,2014-07-30,2.575342,39.0
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.091605,2014-08-05,2.421918,39.0
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546,2018-12-09,6.013699,33.0
4,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090834,2015-12-02,3.452055,33.0
5,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.090355,2014-02-28,2.153425,26.0


In [97]:
# 0 and 1 year prediction window matched case and control
df_matched_case_control_CRC_01yr = df_matched_case_control_CRC[(df_matched_case_control_CRC.FENC2idx_years > 2)]
df_matched_case_control_CRC_01yr.shape
df_matched_case_control_CRC_01yr.head()
df_matched_case_control_CRC_01yr.groupby('has_CRC').size()

(5076, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2014-07-30,Black,Non-Hispanic,1,1974,0.091605,2014-07-30,2.575342,39.0
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.091605,2014-08-05,2.421918,39.0
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546,2018-12-09,6.013699,33.0
4,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090834,2015-12-02,3.452055,33.0
5,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.090355,2014-02-28,2.153425,26.0


has_CRC
0.0    4328
1.0     748
dtype: int64

In [98]:
df_matched_case_control_CC_01yr = df_matched_case_control_CC[(df_matched_case_control_CC.FENC2idx_years > 2)]
df_matched_case_control_CC_01yr.shape
df_matched_case_control_CC_01yr.head()
df_matched_case_control_CC_01yr.groupby('has_CRC').size()

(4103, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060d2b34f18907a0050569ea8fb,0.0,NaT,NaN,2012-01-02,585.6,2012-01-30,585.6,1974-09-01,1,03,N,2015-04-17,Black,Non-Hispanic,1,1974,0.091615,2015-04-17,3.290411,40.0
2,11e75060db07ec1ebb6a0050569ea8fb,0.0,NaT,NaN,2012-06-20,7840,2012-08-08,78652,1982-10-11,0,03,N,2015-12-02,Black,Non-Hispanic,1,1982,0.090844,2015-12-02,3.452055,33.0
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090940,2020-01-30,8.035616,38.0
5,11e75061126eaa76a7850050569ea8fb,1.0,2015-05-26,153.9,2012-06-01,726.90,2012-07-26,719.46,1966-03-26,0,05,N,NaT,White,Non-Hispanic,3,1966,0.092391,2015-05-26,2.983562,49.0
7,11e75060e4aded54907a0050569ea8fb,0.0,NaT,NaN,2012-01-10,530.81,2012-01-13,465.9,1986-03-13,0,05,N,2016-10-04,White,Non-Hispanic,3,1986,0.090460,2016-10-04,4.736986,30.0


has_CRC
0.0    3492
1.0     611
dtype: int64

In [99]:
df_matched_case_control_RC_01yr = df_matched_case_control_RC[(df_matched_case_control_RC.FENC2idx_years > 2)]
df_matched_case_control_RC_01yr.shape
df_matched_case_control_RC_01yr.head()
df_matched_case_control_RC_01yr.groupby('has_CRC').size()

(1751, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,0,03,N,2019-05-15,Black,Non-Hispanic,1,1979,0.084759,2019-05-15,6.890411,40.0
1,11e75060ae9cfb10bc3a0050569ea8fb,0.0,NaT,NaN,2012-03-04,648.93,2012-03-12,640.03,1974-12-25,0,03,N,2014-08-05,Black,Non-Hispanic,1,1974,0.085241,2014-08-05,2.421918,39.0
4,11e7506134d2ba128d0b0050569ea8fb,0.0,NaT,NaN,2012-01-04,474.02,2012-01-06,474.00,1987-10-09,0,05,N,2014-02-28,White,Non-Hispanic,3,1987,0.095986,2014-02-28,2.153425,26.0
6,11e75060edac71e6a7850050569ea8fb,1.0,2015-03-27,154.0,2012-01-01,585.6,2012-01-02,585.6,1966-02-27,0,03,N,NaT,Black,Non-Hispanic,1,1966,0.086017,2015-03-27,3.235616,49.0
7,11e750612fc38484ab480050569ea8fb,0.0,NaT,NaN,2012-02-28,356.0,2012-03-12,359.21,1985-07-21,1,05,N,2018-12-18,White,Non-Hispanic,3,1985,0.098430,2018-12-18,6.808219,33.0


has_CRC
0.0    1489
1.0     262
dtype: int64

In [100]:
os.chdir('/mnt/data1/chong/2021-CRC/psm_result')

In [101]:
df_matched_case_control_CRC_01yr.to_csv('matched_case_control_CRC_01yr.csv')
df_matched_case_control_CC_01yr.to_csv('matched_case_control_CC_01yr.csv')
df_matched_case_control_RC_01yr.to_csv('matched_case_control_RC_01yr.csv')


df_matched_control_CRC.to_csv('matched_control_CRC.csv')
df_matched_control_CC.to_csv('matched_control_CC.csv')
df_matched_control_RC.to_csv('matched_control_RC.csv')

In [102]:
# 3 year prediction window matched case and control
df_matched_case_control_CRC_3yr = df_matched_case_control_CRC[(df_matched_case_control_CRC.FENC2idx_years > 4)]
df_matched_case_control_CRC_3yr.shape
df_matched_case_control_CRC_3yr.head()
df_matched_case_control_CRC_3yr.groupby('has_CRC').size()

(2729, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546,2018-12-09,6.013699,33.0
6,11e75060a5363e2e8e960050569ea8fb,0.0,NaT,NaN,2012-01-03,295.75,2012-01-06,295.75,1973-12-21,0,05,N,2017-05-10,White,Non-Hispanic,3,1973,0.091701,2017-05-10,5.353425,43.0
8,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090930,2020-01-30,8.035616,38.0
11,11e75060acc9d830845d0050569ea8fb,0.0,NaT,NaN,2012-09-14,305.1,2013-01-03,784.0,1984-05-16,0,03,N,2017-08-08,Black,Non-Hispanic,1,1984,0.090642,2017-08-08,4.901370,33.0
12,11e75060e4aded54907a0050569ea8fb,0.0,NaT,NaN,2012-01-10,530.81,2012-01-13,465.9,1986-03-13,0,05,N,2016-10-04,White,Non-Hispanic,3,1986,0.090451,2016-10-04,4.736986,30.0


has_CRC
0.0    2327
1.0     402
dtype: int64

In [103]:
df_matched_case_control_CC_3yr = df_matched_case_control_CC[(df_matched_case_control_CC.FENC2idx_years > 4)]
df_matched_case_control_CC_3yr.shape
df_matched_case_control_CC_3yr.head()
df_matched_case_control_CC_3yr.groupby('has_CRC').size()

(2236, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090940,2020-01-30,8.035616,38.0
7,11e75060e4aded54907a0050569ea8fb,0.0,NaT,NaN,2012-01-10,530.81,2012-01-13,465.9,1986-03-13,0,05,N,2016-10-04,White,Non-Hispanic,3,1986,0.090460,2016-10-04,4.736986,30.0
12,11e75060af8bbd5eab480050569ea8fb,0.0,NaT,NaN,2012-01-06,786.50,2012-01-27,278.00,1973-12-03,0,03,N,2019-04-02,Black,Non-Hispanic,1,1973,0.091711,2019-04-02,7.241096,45.0
14,11e75061023aa5ceab480050569ea8fb,0.0,NaT,NaN,2012-09-04,599.0,2012-10-10,524.60,1980-07-23,0,05,N,2017-04-20,White,Non-Hispanic,3,1980,0.091036,2017-04-20,4.627397,36.0
16,11e75060abfb056494360050569ea8fb,0.0,NaT,NaN,2012-02-13,366.16,2012-03-04,250.01,1971-11-15,0,03,N,2016-11-11,Black,Non-Hispanic,1,1971,0.091905,2016-11-11,4.747945,44.0


has_CRC
0.0    1900
1.0     336
dtype: int64

In [104]:
df_matched_case_control_RC_3yr = df_matched_case_control_RC[(df_matched_case_control_RC.FENC2idx_years > 4)]
df_matched_case_control_RC_3yr.shape
df_matched_case_control_RC_3yr.head()
df_matched_case_control_RC_3yr.groupby('has_CRC').size()

(1021, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,0,03,N,2019-05-15,Black,Non-Hispanic,1,1979,0.084759,2019-05-15,6.890411,40.0
7,11e750612fc38484ab480050569ea8fb,0.0,NaT,NaN,2012-02-28,356.0,2012-03-12,359.21,1985-07-21,1,05,N,2018-12-18,White,Non-Hispanic,3,1985,0.098430,2018-12-18,6.808219,33.0
10,11e75060c2527d1088ed0050569ea8fb,0.0,NaT,NaN,2012-03-10,250.00,2012-03-16,250.00,1975-04-21,0,03,N,2017-11-14,Black,Non-Hispanic,1,1975,0.085145,2017-11-14,5.684932,42.0
14,11e750609a7efeb28e960050569ea8fb,0.0,NaT,NaN,2012-05-21,823.01,2012-05-23,E968.2,1977-10-03,1,03,N,2016-05-25,Black,Non-Hispanic,1,1977,0.086945,2016-05-25,4.013699,38.0
18,11e75060ff2cf9feba790050569ea8fb,0.0,NaT,NaN,2012-01-10,345.90,2012-04-12,780.39,1984-08-20,0,03,N,2018-07-28,Black,Non-Hispanic,1,1984,0.084280,2018-07-28,6.550685,33.0


has_CRC
0.0    882
1.0    139
dtype: int64

In [105]:
df_matched_case_control_CRC_3yr.to_csv('matched_case_control_CRC_3yr.csv')
df_matched_case_control_CC_3yr.to_csv('matched_case_control_CC_3yr.csv')
df_matched_case_control_RC_3yr.to_csv('matched_case_control_RC_3yr.csv')

In [106]:
# 5 year prediction window matched case and control

df_matched_case_control_CRC_5yr = df_matched_case_control_CRC[(df_matched_case_control_CRC.FENC2idx_years > 6)]
df_matched_case_control_CRC_5yr.shape
df_matched_case_control_CRC_5yr.head()
df_matched_case_control_CRC_5yr.groupby('has_CRC').size()

(1068, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
2,11e750614041a6c494250050569ea8fb,0.0,NaT,NaN,2012-12-05,789.09,2012-12-12,590.10,1985-07-22,0,05,N,2018-12-09,White,Non-Hispanic,3,1985,0.090546,2018-12-09,6.013699,33.0
8,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090930,2020-01-30,8.035616,38.0
17,11e75060af8bbd5eab480050569ea8fb,0.0,NaT,NaN,2012-01-06,786.50,2012-01-27,278.00,1973-12-03,0,03,N,2019-04-02,Black,Non-Hispanic,1,1973,0.091702,2019-04-02,7.241096,45.0
20,11e750612fc38484ab480050569ea8fb,0.0,NaT,NaN,2012-02-28,356.0,2012-03-12,359.21,1985-07-21,1,05,N,2018-12-18,White,Non-Hispanic,3,1985,0.090546,2018-12-18,6.808219,33.0
23,11e75060abfb056494360050569ea8fb,0.0,NaT,NaN,2012-02-13,366.16,2012-03-04,250.01,1971-11-15,0,03,N,2018-05-01,Black,Non-Hispanic,1,1971,0.091895,2018-05-01,6.216438,46.0


has_CRC
0.0    927
1.0    141
dtype: int64

In [107]:
df_matched_case_control_CC_5yr = df_matched_case_control_CC[(df_matched_case_control_CC.FENC2idx_years > 6)]
df_matched_case_control_CC_5yr.shape
df_matched_case_control_CC_5yr.head()
df_matched_case_control_CC_5yr.groupby('has_CRC').size()

(892, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
4,11e750610fb3ce06bc3a0050569ea8fb,1.0,2020-01-30,C18.9,2012-01-19,250.00,2012-03-27,368.8,1981-05-25,0,03,N,NaT,Black,Non-Hispanic,1,1981,0.090940,2020-01-30,8.035616,38.0
12,11e75060af8bbd5eab480050569ea8fb,0.0,NaT,NaN,2012-01-06,786.50,2012-01-27,278.00,1973-12-03,0,03,N,2019-04-02,Black,Non-Hispanic,1,1973,0.091711,2019-04-02,7.241096,45.0
18,11e75060a6aa55609be90050569ea8fb,0.0,NaT,NaN,2012-01-03,295.30,2012-03-02,295.30,1970-01-12,0,03,N,2019-05-01,Black,Non-Hispanic,1,1970,0.092002,2019-05-01,7.328767,49.0
48,11e750611fcf32c694250050569ea8fb,0.0,NaT,NaN,2012-01-02,799.3,2012-01-09,799.3,1989-01-05,0,05,N,2019-03-25,White,Non-Hispanic,3,1989,0.090174,2019-03-25,7.230137,30.0
53,11e75060d74e55b8baf60050569ea8fb,0.0,NaT,NaN,2012-01-04,648.23,2012-01-19,V28.3,1981-08-24,0,03,N,2019-10-30,Black,Non-Hispanic,1,1981,0.090940,2019-10-30,7.824658,38.0


has_CRC
0.0    773
1.0    119
dtype: int64

In [108]:
df_matched_case_control_RC_5yr = df_matched_case_control_RC[(df_matched_case_control_RC.FENC2idx_years > 6)]
df_matched_case_control_RC_5yr.shape
df_matched_case_control_RC_5yr.head()
df_matched_case_control_RC_5yr.groupby('has_CRC').size()

(466, 21)

,PATID,has_CRC,CRC_f_date,CRC_f_code,AllENC_f_date,AllENC_f_code,AllENC_2nd_date,AllENC_2nd_code,BIRTH_DATE,SEX,RACE,HISPANIC,ref_index_date,race_c3,ethnicity,race_ethnicity,BIRTH_YEAR,pscore,index_CRC,FENC2idx_years,age_index
0,11e75060b2681860ad370050569ea8fb,0.0,NaT,NaN,2012-06-25,815.00,2013-02-27,V72.0,1979-03-15,0,03,N,2019-05-15,Black,Non-Hispanic,1,1979,0.084759,2019-05-15,6.890411,40.0
7,11e750612fc38484ab480050569ea8fb,0.0,NaT,NaN,2012-02-28,356.0,2012-03-12,359.21,1985-07-21,1,05,N,2018-12-18,White,Non-Hispanic,3,1985,0.098430,2018-12-18,6.808219,33.0
18,11e75060ff2cf9feba790050569ea8fb,0.0,NaT,NaN,2012-01-10,345.90,2012-04-12,780.39,1984-08-20,0,03,N,2018-07-28,Black,Non-Hispanic,1,1984,0.084280,2018-07-28,6.550685,33.0
19,11e750611fcf32c694250050569ea8fb,0.0,NaT,NaN,2012-01-02,799.3,2012-01-09,799.3,1989-01-05,0,05,N,2019-11-05,White,Non-Hispanic,3,1989,0.095771,2019-11-05,7.846575,30.0
20,11e750609911ae628e960050569ea8fb,0.0,NaT,NaN,2012-01-25,300.00,2012-07-17,300.00,1975-06-23,0,05,N,2018-11-16,White,Non-Hispanic,3,1975,0.097284,2018-11-16,6.813699,43.0


has_CRC
0.0    404
1.0     62
dtype: int64

In [109]:
df_matched_case_control_CRC_5yr.to_csv('matched_case_control_CRC_5yr.csv')
df_matched_case_control_CC_5yr.to_csv('matched_case_control_CC_5yr.csv')
df_matched_case_control_RC_5yr.to_csv('matched_case_control_RC_5yr.csv')